In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from utils import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [2]:
train, test = load_data()

In [3]:
X, y = calculate_target(train, 0)

In [4]:
X.head()

,time,date,v_l,q,n_tr,sum_b,code_azs,id,first_prch,location,region,code,code1,percent,type
3,08:24:24,2017-10-23,31.01,0,2017,1181.45,5915.0,21-8B2-7695,27.12.16 23:06:11,10.0,454.5,1010121,126.0,0.0,7
4,NaN,2017-03-19,51.95,0,2017,1818.09,2418.0,21-8B2-7695,27.12.16 23:06:11,9.0,454.0,1010121,126.0,0.0,5
5,19:45:02,2017-08-02,48.64,0,2017,1746.05,2546.0,21-8B2-7695,27.12.16 23:06:11,9.0,453.5,1010121,126.0,0.0,7
6,NaN,2017-05-08,17.43,0,2017,636.10,11152.0,21-8B2-7695,27.12.16 23:06:11,10.0,454.5,1010121,126.0,0.0,13
7,NaN,2017-09-07,23.77,0,2017,908.12,5915.0,21-8B2-7695,27.12.16 23:06:11,10.0,454.5,1010121,126.0,0.0,13


In [5]:
def unique_cnt(series):
    return len(series.unique())

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y )

In [7]:
X_train_agg = X_train.groupby('id')[['v_l', 'q', 'sum_b', 'location', 'code', 'percent', 'type']].agg({
    'v_l':['min', 'max', 'median', 'sum'],
    'q':['min', 'max', 'median', 'sum'],
    'sum_b':['min', 'max', 'median', 'sum'],
    'location':unique_cnt,
    'code':unique_cnt,
    'percent':['min', 'max', 'median', 'sum'],
    'type':unique_cnt
})

In [8]:
X_train_agg.head()

v_l                          q                  sum_b           \
               min    max median      sum min max median sum     min      max   
id                                                                              
21-8A1-14021  8.28  38.18  25.39  1942.07   0   0    0.0   0  272.47  1451.13   
21-8A1-14023  0.00  48.61  13.45   715.12   0   1    0.0  11   53.64  1817.98   
21-8A1-1403   0.00  59.55  36.36  3499.47   0   4    0.0  28   26.36  2272.65   
21-8A1-14032  0.00  47.85  14.00  1416.04   0   2    0.0   5   77.27  1665.02   
21-8A1-14038  0.00  54.45  13.45  2096.19   0   1    0.0  18    0.32  1965.81   

                                   location       code percent           \
               median        sum unique_cnt unique_cnt     min      max   
id                                                                        
21-8A1-14021   908.97   69830.48        4.0          3     0.0   491.75   
21-8A1-14023   454.45   25437.69        4.0         12     0.0    27.28   
21-8A1-1403   1363.42  131479.97        5.0         15     0.0  2272.65   
21-8A1-14032   499.72   47424.84        3.0          6     0.0   305.91   
21-8A1-14038   454.55   75453.33        5.0          6     0.0   454.31   

                                    type  
             median       sum unique_cnt  
id                                        
21-8A1-14021   0.00   1409.31          3  
21-8A1-14023   0.00    272.70          5  
21-8A1-1403    0.05  10535.58          8  
21-8A1-14032  12.24   2021.04          5  
21-8A1-14038   0.00   2440.88          6

In [9]:
rf = RandomForestClassifier(n_estimators=100, n_jobs = -2, random_state=42)

In [10]:
rf.fit(X_train_agg, y_train.sort_index())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-2,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [11]:
X_test_agg = X_test.groupby('id')[['v_l', 'q', 'sum_b', 'location', 'code', 'percent', 'type']].agg({
    'v_l':['min', 'max', 'median', 'sum'],
    'q':['min', 'max', 'median', 'sum'],
    'sum_b':['min', 'max', 'median', 'sum'],
    'location':unique_cnt,
    'code':unique_cnt,
    'percent':['min', 'max', 'median', 'sum'],
    'type':unique_cnt
})

In [12]:
preds = rf.predict_proba(X_test_agg)

In [13]:
X_test_agg.head()

v_l                          q                 sum_b           \
              min    max median      sum min max median sum    min      max   
id                                                                            
21-186G-1161  0.0  57.27   7.53   460.57   0   1    0.0  10  26.36  2199.27   
21-186G-1172  0.0  25.33   9.36   719.02   0   1    0.0   1  90.91   909.25   
21-186G-1190  0.0  19.71   7.93   340.13   0   1    0.0   3  35.45   727.26   
21-186G-1193  0.0  27.73  10.66   397.06   0   1    0.0   2  35.45  1025.91   
21-186G-12    0.0  45.45  22.72  2452.22   0   1    0.0  17  40.91  1666.71   

                                 location       code percent                  \
              median       sum unique_cnt unique_cnt     min      max median   
id                                                                             
21-186G-1161  272.49  18283.55        3.0          7     0.0    58.34  0.000   
21-186G-1172  340.49  25681.58        4.0          3     0.0   227.00  0.000   
21-186G-1190  272.89  12189.73        4.0          4     0.0   454.47  0.025   
21-186G-1193  363.63  13927.99        4.0          3     0.0   790.91  0.000   
21-186G-12    827.15  89850.16        6.0          8     0.0  1405.82  0.000   

                            type  
                  sum unique_cnt  
id                                
21-186G-1161   198.23          2  
21-186G-1172   231.31          3  
21-186G-1190   510.25          3  
21-186G-1193   965.26          5  
21-186G-12    3072.88          7

In [14]:
preds.shape

(18264, 2)

In [15]:
y_test.shape

(18264,)

In [16]:
roc_auc_score(y_test.sort_index(), preds[:, 1])

0.82009468488362058

# Let's make a prediction!

In [17]:
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [18]:
sample_submission.sort_values(by = 'id')

,id,proba
0,21-186G-1142,0.5
15307,21-186G-1225,0.5
1,21-186G-1227,0.5
15308,21-186G-1232,0.5
15309,21-186G-1258,0.5
15310,21-186G-1287,0.5
2,21-186G-1306,0.5
3,21-186G-1310,0.5
15311,21-186G-1344,0.5
4,21-186G-1358,0.5


In [20]:
X_agg = X.groupby('id')[['v_l', 'q', 'sum_b', 'location', 'code', 'percent', 'type']].agg({
    'v_l':['min', 'max', 'median', 'sum'],
    'q':['min', 'max', 'median', 'sum'],
    'sum_b':['min', 'max', 'median', 'sum'],
    'location':unique_cnt,
    'code':unique_cnt,
    'percent':['min', 'max', 'median', 'sum'],
    'type':unique_cnt
})

In [21]:
rf.fit(X_agg, y.sort_index())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-2,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [22]:
test_agg = test.groupby('id')[['v_l', 'q', 'sum_b', 'location', 'code', 'percent', 'type']].agg({
    'v_l':['min', 'max', 'median', 'sum'],
    'q':['min', 'max', 'median', 'sum'],
    'sum_b':['min', 'max', 'median', 'sum'],
    'location':unique_cnt,
    'code':unique_cnt,
    'percent':['min', 'max', 'median', 'sum'],
    'type':unique_cnt
})

In [23]:
test_agg.head()

v_l                           q                  sum_b  \
               min    max  median      sum min max median sum     min   
id                                                                      
21-186G-1142  0.00  57.04  25.705  1255.58   0   1    0.0  22   31.82   
21-186G-1225  0.00  32.50   0.000   544.48   0   1    1.0  41   31.82   
21-186G-1227  4.13  24.47  10.680   439.88   0   0    0.0   0  154.36   
21-186G-1232  0.00  53.93  27.270  1532.83   0   1    0.0   3   53.64   
21-186G-1258  0.00  60.00  22.985  1669.30   0   1    0.0   8   13.64   

                                           location       code percent  \
                  max   median       sum unique_cnt unique_cnt     min   
id                                                                       
21-186G-1142  2227.27   909.08  46867.86        2.0         12     0.0   
21-186G-1225  1330.87   135.45  24726.12        4.0         15     0.0   
21-186G-1227   909.16   363.80  16137.20        2.0          4     0.0   
21-186G-1232  1854.55  1066.09  57665.19        3.0         11     0.0   
21-186G-1258  2087.40   908.97  64477.12        6.0         12     0.0   

                                            type  
                  max median      sum unique_cnt  
id                                                
21-186G-1142   974.08    0.0  2310.34          5  
21-186G-1225   135.45    0.0   630.90          5  
21-186G-1227   363.49    0.0   629.16          5  
21-186G-1232  1043.64    0.0  3375.81          6  
21-186G-1258   909.09    0.0  3196.04          7

In [24]:
test_agg.shape

(31122, 19)

In [25]:
preds = rf.predict_proba(test_agg)

In [28]:
predictions = pd.Series(data = preds[:, 1], index = test_agg.index)

In [30]:
predictions = predictions.reindex(index = sample_submission.id)

In [33]:
sample_submission.proba = predictions.values

In [34]:
sample_submission.head()

,id,proba
0,21-186G-1142,0.02
1,21-186G-1227,0.04
2,21-186G-1306,0.00
3,21-186G-1310,0.00
4,21-186G-1358,0.00


In [35]:
sample_submission.to_csv('rf_baseline_submission.csv', index = False)

In [36]:
check = pd.read_csv('./rf_baseline_submission.csv')

In [39]:
check.head()

,id,proba
0,21-186G-1142,0.02
1,21-186G-1227,0.04
2,21-186G-1306,0.00
3,21-186G-1310,0.00
4,21-186G-1358,0.00
